In [1]:
import math
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

import numpy as np
from astropy.io import fits

import scipy.interpolate as spi
from sklearn.preprocessing import MinMaxScaler
import cv2

from pathlib import Path
from tqdm import tqdm, trange

In [2]:
DB = {
    4471 : 'He I'
};

DZ = {
    3933.7:'Ca II K',
    3968.5:'Ca II H',
};

In [11]:
def main(path_dir, 
         check_lines_dict,
         delta = 3.5e-06,
         center_broading = 5,
         peripheral_broading = 15,
         sn = 10):
    check_lines = [list(check_lines_dict.keys()) ,list(check_lines_dict.values())]
    name = []
    path = []
    for root,dirs,files in os.walk(path_dir):
        for file in files:
            temp = os.path.join(root,file)
            path.append(temp)
    for i in trange(0,len(path)):
        p = Lines_check(path[i], 
                       check_lines, 
                       len(Path(path_dir).parts), 
                       delta = delta, 
                       center_broading = center_broading, 
                       peripheral_broading = peripheral_broading,
                       sn = sn)
        if p != -1:
            name.append(p)
    return name

In [12]:
def Delta_cal(x,
              y,
              line,
              center_broading,
              peripheral_broading
             ):
    ave_c = sum(y[(line - center_broading < x)&(x < line + center_broading)]) / len(y[(line - center_broading < x)&(x < line + center_broading)])
            
    ave_p = ((sum(y[(line + center_broading < x)&(x < line + center_broading + peripheral_broading)]) 
             + sum(y[(line - center_broading - peripheral_broading < x)&(x < line - center_broading)])) 
             / (len(y[(line + center_broading < x)&(x < line + center_broading + peripheral_broading)]) 
                + len(y[(line - center_broading - peripheral_broading < x)&(x < line - center_broading)])))
    return (ave_p - ave_c) / (max(y) - min(y))

In [13]:
def Lines_check(file,
                lines,
                a,
                delta,
                center_broading,
                peripheral_broading,
                sn = 10
               ):
    file_fits = fits.open(file)
    x = file_fits[1].data[0].field('WAVELENGTH')
    y = file_fits[1].data[0].field('FLUX')
    z = file_fits[0].header['Z']
    sng = file_fits[0].header['SNRU']
    subclass = file_fits[0].header['SUBCLASS']
    file_fits.close()
    
    ipo = spi.splrep(x , y , k = 3)
    X = x
    Y = spi.splev(X, ipo)
    Y = cv2.GaussianBlur(src = Y, ksize = (29, 29), sigmaX = 5)
    
    if sng > sn:
        line = []
        if z == -9999:
            z = 0
        for i in range(0,len(lines[0])):
            line.append(lines[0][i] / (z + 1))
        text = Path(file).parts[a] + '/' + Path(file).parts[a+1] + '/' + file_fits[0].header['FILENAME']
        flag = 0
        n = 0

        for i in range(0,len(line)):
            if min(x) <= line[i] <= max(x):
                n = n + 1
                delta0 = Delta_cal(x, y, line[i], center_broading, peripheral_broading)
                delta1 = Delta_cal(X, Y, line[i], center_broading, peripheral_broading)
                if delta0 >= delta and delta1 >= delta:
                    flag = flag + 1
        if flag == n and n != 0:
            return text
        else:
            return -1
    else:
        return -1

In [14]:
name = main(r'C:\Users\Administrator\Desktop\大创\Data\temp\test' , DZ, sn = 0)

100%|██████████| 9270/9270 [05:10<00:00, 29.88it/s]  


In [15]:
print(len(name))
name

2367


['WDJ000158.51+413205.94/271005105/spec-56985-M31000N42B2_sp05-105.fits',
 'WDJ000215.38+073359.56/369908226/spec-57309-EG000015N080026M01_sp08-226.fits',
 'WDJ000228.29+312908.27/616406127/spec-58102-HD235310N315436V01_sp06-127.fits',
 'WDJ000358.79+433601.89/615001162/spec-58099-GACII001N45B1_sp01-162.fits',
 'WDJ000414.69+404736.53/474614139/spec-57694-M31003N39M1_sp14-139.fits',
 'WDJ000414.69+404736.53/476512106/spec-57697-M31358N39B1_sp12-106.fits',
 'WDJ000414.69+404736.53/483412106/spec-57716-M31358N39M1_sp12-106.fits',
 'WDJ000414.69+404736.53/501614139/spec-57750-M31003N39B1_sp14-139.fits',
 'WDJ000448.34+343628.10/500602106/spec-57747-M31002N36B2_sp02-106.fits',
 'WDJ000501.13+400334.23/474603051/spec-57694-M31003N39M1_sp03-051.fits',
 'WDJ000501.13+400334.23/483313010/spec-57716-M31358N39B2_sp13-010.fits',
 'WDJ000501.13+400334.23/501603051/spec-57750-M31003N39B1_sp03-051.fits',
 'WDJ000509.94+003809.62/369207237/spec-57308-EG000023N024031M01_sp07-237.fits',
 'WDJ000704.58+